In [2]:

%pip install pylablib

Defaulting to user installation because normal site-packages is not writeable
  Using cached pylablib-1.4.3-cp311-cp311-win_amd64.whl.metadata (6.3 kB)
  Using cached rpyc-6.0.0-py3-none-any.whl.metadata (3.5 kB)
  Using cached pyft232-0.12-py3-none-any.whl
  Using cached PyVISA-1.14.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pyserial-3.5-py2.py3-none-any.whl.metadata (1.6 kB)
  Using cached pyusb-1.2.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached pyqtgraph-0.13.7-py3-none-any.whl.metadata (1.3 kB)
  Using cached plumbum-1.8.3-py3-none-any.whl.metadata (10 kB)
Using cached pylablib-1.4.3-cp311-cp311-win_amd64.whl (1.8 MB)
Using cached PyVISA-1.14.1-py3-none-any.whl (176 kB)
Using cached pyserial-3.5-py2.py3-none-any.whl (90 kB)
Using cached pyusb-1.2.1-py3-none-any.whl (58 kB)
Using cached pyqtgraph-0.13.7-py3-none-any.whl (1.9 MB)
Using cached rpyc-6.0.0-py3-none-any.whl (74 kB)
Using cached plumbum-1.8.3-py3-none-any.whl (127 kB)
Note: you may need to restart the kernel

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [5]:
##importing all necessary shit.
import numpy as np
import scipy
from sklearn import linear_model
from pylablib.devices import Newport
from numpy.polynomial import polynomial as P
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures


In [6]:
Newport.get_usb_devices_number_picomotor()
##Lets you know whether importing was successful and whether you can access your USB port

0

In [ ]:
""" At this point, USB permissions are likely going to be an issue. Took me literally 2 full days to resolve, but the soln is (for Windows):
    1.) Elevate all USB permissions in REGEDIT. You can navigate to USB permissions by going HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Enum, 
        and USB should be at the bottom of this. Right click the folder and go down to permissions, then just elevate all that shit.
    2.) Run Python in administrator if possible. I think this gets rid of the premissions issue but, regardless, I couldn't "fix" it on my end until
        I did both. It's possible that is because I didn't restart my system after editing REGEDIT until I also ran as Admin, so idk which one fixed
        it. Just do both. """
stage1 = Newport.Picomotor8742() ##Will connect 8742 if it is in first USB in system. If connected via Ethernet, put IP Address or Host Name in ()
                                 ##i.e: Picomotor8742("0.0.0.0") or Picomotor8742("8080-123456")

locationDataX1 = [0] 
locationDataY1 = [0]
locationDataX2 = [0]
locationDataY2 =[0]
locationDataAll = [[0,0,0,0]]
efficiencyData = []
efficiencyData.append(-1*float(input("What is the power meter reading now?")))
stage1.set_position_reference() ##Sets current position as reference position so motor can go back. Note that since this is an open-loop motor, going
                                ## back won't be perfect, but we don't need it to be.

In [7]:
##Now Setting up manual input for SGD
def randPointData(axes):
    for i in axes:
        newPosition = np.random.randint(-500, 500)
        stage1.move_to(i+1, newPosition)
        stage1.wait_move()       
    newEfficiency = input("Input the new power meter reading here")
    efficiencyData.append(-1*float(newEfficiency))
    locationDataX1.append(stage1.get_position(axis=1))
    locationDataY1.append(stage1.get_position(axis=2))
    locationDataX2.append(stage1.get_position(axis=3))
    locationDataY2.append(stage1.get_position(axis=4))
    locationDataAll.append(stage1.get_position())

def getRandPoints(numOfPoints, axes):
    for i in numOfPoints:
        return randPointData(axes)

##Now we can manually gather data regarding efficiency and store it to be regressed ~~stochastically~~

        

In [9]:
##Making the model
def fittingData(locationDataAll, efficiencyData):
    polyDeg = PolynomialFeatures(2)
    locationDataNew = polyDeg.fit_transform(polyDeg)
    model = LinearRegression()
    newModel = model.fit(locationDataNew, efficiencyData)
    optimumStep = scipy.optimize.basinhopping(newModel)
    return("You should move to: " + optimumStep)

##Finding the coefficients for the most optimal mirror placement

    


In [ ]:
getRandPoints(10, 1)
print(locationDataAll)
print(efficiencyData)
fittingData(locationDataAll, efficiencyData)